# Aula 18 - Introdução ao Aprendizado Supervisionado: Regressão

Nesta aula vamos estudar o uso de algoritmos de **Aprendizado de Máquina Supervisionado**. Focaremos nos algoritmos utilizados para **regressão**, que é a tarefa de predizer valores de saída (dados numéricos) para conjuntos de dados de entrada (numéricos ou transformados em numéricos).

Existem muitos algoritmos para se realizar essa tarefa, entre eles:
- Regressão Linear 
- Redes Neurais
- Árvores de Decisão
- RandomForest
- Support Vector Regression (SVR)

Cada uma dessas técnicas tem uma forte base matemática e conceitual. Os algoritmos foram propostos ao longo de diversas décadas e possuem diferentes cenários de aplicações. Não teremos tempo de ver cada um deles a fundo, nem mesmo todos eles. Iremos nos focar nos procedimentos básicos para se utilizar esses algoritmos.

--------------------------

## Exemplo 1 - Preço de Casas

![image-3.png](attachment:image-3.png)

Foram medidos os preços de casas vendidas entre Maio de 2014 e Maio de 2015 em Seattle. Foram anotados 21 atributos descritivos destas casas, como número de quartos e número de banheiros, entre outros.

Os dados estão diponíveis em: https://www.openml.org/d/42731

Será que é possível usar as características das casas para predizer qual deveria ser o seu valor de venda?

### Exemplo 1 - Passo zero: carregar os dados

In [ ]:
# Para trabalhar com dados em tabelas
import pandas as pd

# Para trabalhar com gráficos
from sklearn.datasets import fetch_openml

In [ ]:
dados = fetch_openml(data_id=42731)

In [ ]:
dados

In [ ]:
df_casas = pd.DataFrame(data = dados.data, columns=dados.feature_names) # Transformando em dataframe

In [ ]:
df_casas['price'] = dados.target # Colocando a coluna de preços

In [ ]:
df_casas.head()

In [ ]:
df_casas.info() # Observe que não temos nenhum dado faltando para as colunas de informação

### Exemplo 1 - Primeiro passo: separar os dados

Precisamos ter claro em nossas mentes quais são os atributos (colunas) preditivas, isto é, aquelas que usaremos para predizer um atributo alvo (coluna alvo).

Quando se trabalhar com Aprendizado de Máquina Supervisionado, além disso precisamos separar nossos dados em dois conjuntos: um **conjunto de treinamento** e um **conjunto de teste**. Fazemos isso para evitar overfitting do algoritmo.

Existem várias estratégias para se separar dados. Vamos aplicar as principais:

(a) Escolher o tamanho dos conjuntos - 80% para treinamento e 20% para teste;

(b) Selecionar aleatoriamente os dados que irão compor o conjunto de treinamento e teste (usando um seed para garantir reprodutibilidade);

In [ ]:
df_dados = [i for i in range(0,1000)]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
len(df_dados)

1000

In [ ]:
# Escolhendo as colunas preditivas e alvo
x = df_dados[['rank_risco','valor_compra', 'valor_venda','tempo_investido','inflacao']]
y = df_dados['lucro_real']                

# Dividindo conjunto de treinamento e conjunto de teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [ ]:
# Escolhendo as colunas preditivas e alvo
x = df_casas.drop(columns = ['price','zipcode']) # Colunas descritivas
y = df_casas['price']                # Valor alvo

# Dividindo conjunto de treinamento e conjunto de teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

### Exemplo 1 - Segundo passo: treinar o algoritmo 

Vamos usar o conjunto de treinamento para treinar o algoritmo escolhido.

Existem vários algoritmos possíveis. Cada algoritmo tem seus **hiperparâmetros** (parâmetros que devem ser escolhidos para melhorar a performance do algoritmo). Para entender os hiperparâmetros de cada algoritmo é necessário entender a fundo como aquele algoritmo funciona.

Aqui vamos usar um dos algoritmos mais simples existentes, a Regressão Linear:

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression()  # Criamos o objeto do regressor (não mudamos nenhum hiperpârametro)

lr.fit(x_train,y_train) # Treinamos o regressor passando apenas o conjunto de dados de treinamento 

### Exemplo 1 - Terceiro passo: testar e avaliar 

Usando agora o conjunto de teste, iremos testar o classificador criado e treinando no passo anterior.

Uma vez que realizamos um teste, precisamos avaliar o desempenho do nosso método. Diferentes hiperparâmetros e diferentes algoritmos podem ter um desempenho diferente. Apesar de sempre buscarmos o melhor desempenho, devemos levar em consideração outros aspectos para decidir o que é um desempenho aceitável. A primeira coisa a definir são as métricas de desempenho que podem ser comparadas entre diferentes algoritmos. 

Existem várias métricas para se medir o desempenho de um regressor: métricas de performance, métricas de tempo consumido, métricas de memória consumida, etc. Vamos calcular algumas delas:
- MAE
- MSE
- Correlação de Pearson
- $R^2$

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr

In [ ]:
# Perceba que estamos passando apenas o x de teste, afinal o algoritmo é que nos dira qual é o y 
y_predicoes = lr.predict(x_test) 

In [ ]:
MAE = mean_absolute_error(y_true = y_test,      # Gabarito
                          y_pred = y_predicoes) # Respostas do algoritmo ao teste realizado
print('MAE: ', MAE)

In [ ]:
MSE = mean_squared_error(y_true = y_test,      # Gabarito
                         y_pred = y_predicoes) # Respostas do algoritmo ao teste realizado
print('MSE: ', MSE)

A **MAE** e a **MSE** são formas bem simples de avaliar o erro. O problema é como saber se um determinado valor de MAE ou MSE é muito grande? Intuitivamente sabemos que quanto maior esse valor, pior. Entretanto, o valor calculado se refere a escala dos dados, e por isso, pode ser de dificil avaliação. Em geral, devemos comparar duas MAEs ou duas MSEs para dois modelos de regressão diferentes, usando os mesmos dados. Aqueles que tiverem uma MAE ou MSE menores, serão, portanto, melhores.

A escala **R²**, por outro lado, não tem esse problema de comparação. Esse valor já está em uma escala, de forma que sabemos que quanto mais próximo de 1, melhor! Mas atenção, dificilmente um modelo terá R²=1. Então se você obter isso, provavelmente seu modelo está sofrendo de overfitting (ele é muito bom para seus dados, mas não será tão bom para novas entradas). Sempre que você obtiver um valor de R²=1, verifique se não cometeu nenhum erro nos passos anteriores.

In [ ]:
R2 = r2_score(y_true = y_test,      # Gabarito
              y_pred = y_predicoes) # Respostas do algoritmo ao teste realizado
print('R2: ', R2)  # Quanto mais próximos de 1 melhor

O **coeficiente de correlação de Pearson $\rho$** também é mais fácil de interepretar o resultado já que eles está em uma escala de -1 a 1. Quando estamos avaliando o desempenho do modelo, queremos $\rho=1$. 

In [ ]:
rho = pearsonr(y_test, y_predicoes)
print('\u03C1:', rho[0]) # Quanto mais próximo de 1, melhor (OBS: apneas neste caso de utilização)

### Exemplo 1 - O modelo obtido

Nos modelos de Regressão Linear, temos uma função linear de múltiplos coeficientes. Essa função é exatamente o modelo encontrado pelo algoritmo. Para obter os coeficientes, basta fazer:

In [ ]:
a_modelo = lr.coef_      # Coeficientes angulares do modelo (cada um mede a influência de cada coluna)
b_modelo = lr.intercept_ # Coeficiente linear do model

In [ ]:
print(a_modelo, b_modelo)

Durante a etapa de treinamento, métricas de desempenho também são calculadas com o objetivo de se minimizar o erro. Perceba que o R² de treinamento e de teste são diferentes, o que é esperado e desejado. Podemos obter o R² de treinamento, fazendo:

In [ ]:
R2_treinamento = lr.score(x_train, y_train)
print(R2_treinamento)

In [ ]:
y_pred_linear = y_predicoes # Salvando as informações do Modelo Reg Linear para comparar depois

### Voltando no passo 2 e trocando o algoritmo para uma Rede Neural Perceptron Multicamada

Redes Neurais Artificiais como o MLP (Multilayer Perceptron) podem ser altamente sensiveis a escala dos dados (assim como as SVM e o KNN nas tarefas de classificação).

Vamos primeiro realizar o passo intermediário de transformação dos dados: 

In [ ]:
from sklearn.preprocessing import StandardScaler # Importando o Escalonador de Normalização

In [ ]:
# 1.5 - Escalonamento
scaler = StandardScaler() # Criando o objeto de escalonamento
scaler.fit(x_train) # Passando os dados de treinamento para encontrar a escala

x_train_escalonado = scaler.transform(x_train)
x_test_escalonado = scaler.transform(x_test)

Agora vamos criar a rede neural e treinar ela:

In [ ]:
from sklearn.neural_network import MLPRegressor # Importando a Rede Neural MLP para Regressão

In [ ]:
# 2 - Treino
# Instanciando o objeto
ml_perceptron = MLPRegressor(solver='lbfgs', # otimizador por métodos quasi-Newton
                             tol=1e-5, # Limiar para a otimização (treinamento)
                             max_iter=1500, # Limite de iterações durante a otimização (treinamento)
                             random_state=42, # (seed) pesos da rede são inicializados aleatoriamente
                             hidden_layer_sizes=(10,4), # Quantidade de neuronios por camada oculta #(12,7,3)
                             activation='relu') # Função de ativação dos neuronios

ml_perceptron.fit(x_train_escalonado, y_train)  # Treinando (perceba que estamos passando o x_train escalonado)

In [ ]:
# 3 - Teste
y_predicoes = ml_perceptron.predict(x_test_escalonado) # Perceba que estamos passando o x_test escalonado

R2 = r2_score(y_true = y_test,      # Gabarito
              y_pred = y_predicoes) # Respostas do algoritmo ao teste realizado
print('R2: ', R2)  # Quanto mais próximos de 1 melhor

rho = pearsonr(y_test, y_predicoes)
print('\u03C1:', rho[0]) # Quanto mais próximo de 1, melhor (OBS: apneas neste caso de utilização)

MAE = mean_absolute_error(y_true = y_test,      # Gabarito
                          y_pred = y_predicoes) # Respostas do algoritmo ao teste realizado
print('MAE: ', MAE)

MSE = mean_squared_error(y_true = y_test,      # Gabarito
                         y_pred = y_predicoes) # Respostas do algoritmo ao teste realizado
print('MSE: ', MSE)

In [ ]:
y_predicoes_mlp = y_predicoes

#### Gráfico de acerto

Podemos visualizar nossos acertos através de um gráfico de dispersão entre y_test e y_pred. Perceba que se nosso modelo acerta tudo, y_test=y_pred. Dessa forma, teriamos uma reta perfeita com ângulo de inclinação de 45°. Entretanto, se nosso modelo não acertar tudo, então teremos uma outra reta, com outro ângulo de inclinação.

In [ ]:
import seaborn as sns

In [ ]:
ax = sns.regplot(x=y_test,y=y_pred_linear,
                 color="b", scatter_kws={'alpha':0.3}, label='RegLinear') # Regressão Linear
ax = sns.regplot(x=y_test,y=y_predicoes_mlp,
                 color="g", scatter_kws={'alpha':0.3}, label='MLP') # Rede Neural
ax.plot(y_test, y_test, 'r--', linewidth = 2, label='Correto') # Reta 100% correto
ax.set(xlabel='y_test (gabarito)', ylabel='y_pred (respostas do teste)') 
ax.legend()

E se tivessemos escalonado os dados para passar pela Regressão Linear? Qual seria o resultado?

------------------------------
## Exemplo 2 - Fifa 2019 
![image-5.png](attachment:image-5.png)

Contém dados de diversos jogadores de futebol presentes no Jogo Fifa 2019 da EAGames. 

Os dados são públicos no Kaggle em https://www.kaggle.com/karangadiya/fifa19


Queremos prever a performance total do jogador dada pela coluna Overall, usando a idade do jogador (coluna Age), o valor do jogador (coluna Value) e o seu salário (coluna Wage).

### Exemplo 2 - Passo zero: carregar os dados

In [ ]:
import pandas as pd

In [ ]:
# Carregando os dados
dados_fifa = pd.read_csv('data_fifa2019.csv', index_col = 0)

In [ ]:
dados_fifa.head(3)

Vamos olhar como estão os dados das colunas através do .info()

Perceba que as colunas desejadas como preditoras não possuem dados vazios.

In [ ]:
dados_fifa.info()

Vamos converter as colunas Value e Wage que estão em strings para valores numéricos em Euros:

In [ ]:
def convert_currency(col):
    return (col.str.lstrip('€')
               .str.rstrip('.')
               .str.replace('K','000')
               .str.replace('M','000000')
               .astype('float'))

In [ ]:
dados_fifa['Value'] = convert_currency(dados_fifa.Value)
dados_fifa['Wage'] = convert_currency(dados_fifa.Wage)

### Exemplo 2 - Primeiro passo: separar os dados

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Escolhendo as colunas preditivas e alvo
x = dados_fifa[['Age','Value','Wage']] # Selecionando 3 colunas (perceba o uso de dois colchetes)
y = dados_fifa['Overall']                # Valor alvo

# Dividindo conjunto de treinamento e conjunto de teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

### Exemplo 2 - Segundo passo: treinar o modelo

Vamos Escalonar os dados antes de usá-los, realizando assim o passo intermediário de transformação.

In [ ]:
from sklearn.preprocessing import StandardScaler # Importando o Escalonador de Normalização

In [ ]:
# Passo 1.5 - Escalonamento
scaler = StandardScaler() # Criando o objeto de escalonamento
scaler.fit(x_train) # Passando os dados de treinamento para encontrar a escala

x_train_escalonado = scaler.transform(x_train)
x_test_escalonado = scaler.transform(x_test)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression()  # Criamos o objeto do regressor (não mudamos nenhum hiperpârametro)

lr.fit(x_train_escalonado,y_train) # Treinamos o regressor passando apenas o conjunto de dados de treinamento 

### Exemplo 2 - Terceiro passo: testar e avaliar

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
y_predicoes = lr.predict(x_test_escalonado) 

In [ ]:
R2 = r2_score(y_true = y_test,      # Gabarito
              y_pred = y_predicoes) # Respostas do algoritmo ao teste realizado
print('R2: ', R2)  # Quanto mais próximos de 1 melhor

### Transformando as colunas para outras funções matemáticas

Nem sempre a relação entre os dados é puramente linear. Por vezes temos relações polinômias, sinusoidais, entre outras. 

Como podemos adequar nossos dados com relações diferentes para modelos de Regressão Linear?

Primeiro vamos tentar ver como nossos dados se relacionam plotando gráficos de dispersão entre as colunas descritivas e colunas alvo:

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(12, 4))

# Plot de dispersão (scatter)
ax[0].scatter(x['Age'], y, c='blue', alpha=0.5)
ax[1].scatter(x['Value'], y, c='green', alpha=0.5)
ax[2].scatter(x['Wage'], y, c='red', alpha=0.5)

# Aqui estamos explicitamente dando nome aos eixos do gráfico
ax[0].set_xlabel('Age'), ax[0].set_ylabel('Overall')
ax[1].set_xlabel('Value'), ax[1].set_ylabel('Overall')
ax[2].set_xlabel('Wage'), ax[2].set_ylabel('Overall')

# Colocando grid
ax[0].grid(), ax[1].grid(), ax[2].grid()

#ax[1].set_xscale('log')
#ax[2].set_xscale('log')

# Vamos aumentar a separação entre os gráficos
plt.subplots_adjust(wspace = 0.3)

# Aumentando o tamanho da fonte (letra)
plt.rcParams.update({'font.size': 12})

In [ ]:
import numpy as np # O numpy tem funções matemáticas variadas prontas dentro dele
from sklearn.preprocessing import FunctionTransformer  # Módulo para transformar dados por funções

Nossos gráficos sugerem que o Overall depende mais linearmente do logaritmo de Wage e do logaritmo de Value. 

O Numpy tem várias funções matemáticas prontas (https://numpy.org/doc/stable/reference/routines.math.html), entre elas o log10, que tira o logaritmo na base 10 do valor passado. Vamos usar essa função para transformar nossos dados.

In [ ]:
# Criando nosso transformador
transformador = FunctionTransformer(np.log10)

In [ ]:
x1 = dados_fifa['Age']
x2 = transformador.transform(dados_fifa['Value'])
x3 = transformador.transform(dados_fifa['Wage'])

In [ ]:
novos_dados = {'Age':x1,
               'log_Value':x2,
               'log_Wage':x3,
               'Overall':dados_fifa['Overall']
            }

In [ ]:
dados_fifa2 = pd.DataFrame(data=novos_dados)

In [ ]:
dados_fifa2.describe()

O problema de usar a função logaritmica é que ela não é definida para o zero e valores negativos. Dessa forma ela gera valores -inf quando não consegue calcular. Vamos substituir todos os valores -inf por 0.

In [ ]:
dados_fifa2.replace([np.inf, -np.inf], 0, inplace=True)

In [ ]:
dados_fifa2.describe()

In [ ]:
# Passo 1 - Separando dados
x = dados_fifa2.drop(columns=['Overall']) # Selecionando 3 colunas
y = dados_fifa2['Overall']                # Valor alvo
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

# Passo 1.5 - Escalonamento
scaler = StandardScaler() # Criando o objeto de escalonamento
scaler.fit(x_train) # Passando os dados de treinamento para encontrar a escala

x_train_escalonado = scaler.transform(x_train)
x_test_escalonado = scaler.transform(x_test)

# Passo 2 - Treinar
lr = LinearRegression()  # Criamos o objeto do regressor (não mudamos nenhum hiperpârametro)
lr.fit(x_train_escalonado,y_train) # Treinamos o regressor passando apenas o conjunto de dados de treinamento 

# Passo 3 - Avaliar
y_predicoes = lr.predict(x_test_escalonado) 

R2 = r2_score(y_true = y_test,      # Gabarito
              y_pred = y_predicoes) # Respostas do algoritmo ao teste realizado
print('R2: ', R2)  # Quanto mais próximos de 1 melhor

------------------------------
## Exemplo 3 - Fifa2019

Agora queremos repetir o exemplo 2 usando outras colunas de dados, inclusive colunas que não possuem todos os valores para cada jogador.

Pense em colunas que você acredita serem importantes para o modelo. Selecione-as. 

Em seguida, escolha um modelo de Regressão. Pode ser uma Regressão Linear simples, uma Rede Neural Perceptron Multicamada ou uma Regressão Linear modificada para Funções Não Lineares.

Qual foi o R² obtido?

### Exemplo 3 - Passo zero: carregar os dados

In [ ]:
import pandas as pd

In [ ]:
# Carregando os dados
dados_fifa = pd.read_csv('data_fifa2019.csv', index_col = 0)

In [ ]:
dados_fifa.head(3)

In [ ]:
# Arrumando as colunas com de Valor, Salário e Peso para serem colunas numéricas
dados_fifa['Value'] = convert_currency(dados_fifa.Value)
dados_fifa['Wage'] = convert_currency(dados_fifa.Wage)
dados_fifa['Weight'] = dados_fifa.Weight.str.rstrip('lbs').astype('float')


# Selecionando colunas com valores numéricos
colunas_selecionadas = dados_fifa[['Age', 'Potential', 'Value', 'Wage', 'Special', 'International Reputation',
                                   'Weight','HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
                                   'FKAccuracy','LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed',
                                   'Agility', 'Reactions', 'Balance', 'ShotPower', 'Jumping', 'Stamina',
                                   'Strength', 'LongShots', 'GKPositioning','Aggression', 'Interceptions',
                                   'Positioning', 'Vision', 'Penalties','Composure','Marking', 'StandingTackle',
                                   'SlidingTackle', 'GKDiving','GKHandling','GKKicking','GKReflexes', 'Overall']]

In [ ]:
# Excluindo desse meu conjunto de colunas selecionadas as linhas que tem algum valor nulo
df_fifa_selecionados = colunas_selecionadas.dropna()

In [ ]:
df_fifa_selecionados.info()

In [ ]:
df_fifa_selecionados.head()

### Exemplo 3 - Primeiro passo: separar os dados

In [ ]:
## Coloque seu código aqui

### Exemplo 3 - Segundo passo: treinar o modelo

In [ ]:
## Coloque seu código aqui

### Exemplo 3 - Terceiro passo: testar e avaliar

In [ ]:
## Coloque seu código aqui